In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments


tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='wikitext-2/wiki.train.tokens',
    block_size=128
)
    
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

model = GPT2LMHeadModel.from_pretrained('gpt2')


/Users/napoli/anaconda3/envs/311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/napoli/anaconda3/envs/311/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


ValueError: Input file path wikitext-2/wiki.train.tokens not found

In [3]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
# Load the 'wikitext' dataset
wikitext_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')['train']

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def encode(examples):
    return tokenizer(examples['text'])
    
train_dataset = wikitext_dataset.map(encode, batched=True, remove_columns=['text'])



Found cached dataset wikitext (/Users/napoli/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
100%|██████████| 3/3 [00:00<00:00, 813.01it/s]


In [1]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel, AdamW
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Set the batch size and the number of epochs
batch_size = 16
epochs = 3

# Load the 'wikitext' dataset
wikitext_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')['train']

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a padding token and set it to the same as the EOS token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the training_dataset
def encode(examples): return tokenizer(examples['text'])
train_dataset = wikitext_dataset.map( encode, batched = True, remove_columns = ['text']) 

# Build the dataset collator.
data_collator = DataCollatorForLanguageModeling( tokenizer = tokenizer, mlm = False, pad_to_multiple_of = 128 )

# Create a DataLoader
dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, collate_fn = data_collator)

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(epochs):
    print(f'Starting epoch {epoch+1}/{epochs}')
    
    for batch in dataloader:
        # Move the batch tensors to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        print(f'Loss {loss}')

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        # Clear the gradients
        optimizer.zero_grad()

    print(f'Finished epoch {epoch+1}/{epochs}')


/Users/napoli/anaconda3/envs/311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset wikitext (/Users/napoli/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
100%|██████████| 3/3 [00:00<00:00, 632.40it/s]
Loading cached processed dataset at /Users/napoli/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-aaabf8fb94b355e0.arrow
/Users/napoli/anaconda3/envs/311/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this war

Starting epoch 1/3
Loss 3.986405849456787
Loss 3.9257171154022217
Loss 4.255166053771973
Loss 4.06812858581543
Loss 4.010963439941406
Loss 3.7512078285217285


KeyboardInterrupt: 